# Scraped Fixture API

- Get your personal Signal Ocean API subscription key (acquired [here](https://apis.signalocean.com/profile)) and replace it below:

In [1]:
signal_ocean_api_key = '' # replace with your subscription key

## Setup

In [4]:
!pip install signal-ocean

import pandas as pd
from datetime import date, timedelta
import plotly.graph_objects as go
import plotly.express as px


from signal_ocean import Connection
from signal_ocean.scraped_fixtures import ScrapedFixturesAPI

Processing /home/signal_ocean-1.0.7-py3-none-any.whl


## Get Fixtures

In [8]:
def plot_charterers(data, top_k_charterers=3, days = 7):

  data_aggr = data \
              .groupby(['charterer'])['imo'] \
              .unique().reset_index()

  data_aggr['Count Unique Imo'] = data_aggr \
                                            .apply(lambda x: len(x.imo),axis=1)
  top_k = data_aggr \
          .sort_values('Count Unique Imo') \
          .tail(top_k_charterers)
  
  fig = px.bar(top_k, 
               y='charterer', 
               x='Count Unique Imo' ,
               orientation='h',
               title="Top {} charterers, last {} days, VLCC" \
                              .format(top_k_charterers, days)
               )
    
  
  fig.update_layout(
      xaxis_title="Number of fixed Vessels"
  )
  fig.show()

In [6]:
def plot_weekly_fixed_vessels(results):
  freq_dict = {
      0:'W-MON',
      1:'W-TUE',
      2:'W-WED',
      3:'W-THU',
      4:'W-FRI',
      5:'W-SAT',
      6:'W-SUN'
  }
  fixtures_weekly_laycan = results.groupby(
      pd.Grouper(
          key='laycan_from',
          freq = freq_dict[date.today().weekday()],
          closed = 'left',
          label = 'left' 
      )
  )['imo'].unique().reset_index()
  fixtures_weekly_laycan['Count Unique Imo'] = fixtures_weekly_laycan \
                                            .apply(lambda x: len(x.imo),axis=1)

  fig = px.line(
      fixtures_weekly_laycan, 
      title="Port Arrivals per Week",
      x='laycan_from', 
      y='Count Unique Imo', 
      
      # hover_data={"laycan_from": "|%a %d-%m-%Y"},
      # line_dash = 'count distinct IMO'
      )
  fig.show()

## Results

### Get fixtures for last 30 days Ras Tanura/VLCC

In [13]:
%%time

vessel_type_id = 1 # Tanker
period_in_days = 15
vessel_class_id = 84 # VLCC
port_id = 3778 # Ras Tanura

date_from = date.today() - timedelta(days=period_in_days)
date_to = date.today()

# Scraped Fixture API
connection = Connection(signal_ocean_api_key)
api = ScrapedFixturesAPI(connection)
results = pd.DataFrame(api.get_fixtures(
    received_date_from = date_from,
    received_date_to = date_to,
    vessel_type = vessel_type_id,
    port_id = port_id,
    vessel_class_id = vessel_class_id
))
results.head(5)

CPU times: user 28.6 s, sys: 182 ms, total: 28.8 s
Wall time: 2min 59s


### Plot top k charterers

In [11]:
plot_charterers(results, top_k_charterers=3, days=period_in_days)

### Get the weekly number of fixed vessels for Ras Tanura/VLCC

In [12]:
plot_weekly_fixed_vessels(results)